In [1]:
import tensorflow as tf
import json
import numpy as np
import cv2
import pathlib
from sklearn.model_selection import train_test_split

In [ ]:
# labels = tf.data.Dataset.list_files(r'D:\Competitions\ComputerVision\OCR\Signalboard\dataset\train\label\official\*.json')

In [5]:
image_shape = (1024,1024)
image_folder = r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\image\official'
label_folder = r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\label\custom'
validation_ratio = 0.05
random_state = 100
batch_size = 8


# total_data_count = len(list())
# split_count = round(total_data_count * validation_ratio)

# dataset = tf.data.Dataset.list_files(r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\label\custom\*.json',shuffle=False)
# dataset = dataset.map(lambda x:(x,image_path,image_shape),num_parallel_calls=tf.data.AUTOTUNE)
# dataset = dataset.map(read_json,num_parallel_calls=tf.data.AUTOTUNE)

# validation_data = dataset.take(split_count).cache()

# train_data = dataset.skip(split_count).cache()

# dataset = dataset.cache()
# dataset

In [25]:
t,v = Dataset(label_folder,image_folder,batch_size,image_shape).split(validation_ratio,random_state)

In [26]:
image_path, center, ltwh, offset, determine = next(iter(t.take(1)))

In [34]:
x = t.map(lambda image_path, center, ltwh, offset, determine: tf.py_function(
    transform,
    [image_path, center, ltwh, offset, determine],
    [tf.uint8,tf.float64,tf.float64,tf.float64,tf.float64,tf.float64,tf.float64,tf.float64]
))

In [36]:
next(iter(x.take(1)))[1]

<tf.Tensor: shape=(128, 128), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [41]:
(determine>100) & (determine<120)

<tf.Tensor: shape=(14,), dtype=bool, numpy=
array([False,  True, False, False, False, False, False, False, False,
        True,  True,  True,  True,  True])>

In [42]:
determine

<tf.Tensor: shape=(14,), dtype=int32, numpy=
array([848, 119, 122, 124, 126, 304, 219, 142,  25, 114, 109, 108, 115,
       102])>

In [56]:
grid_count = 8
blood = 1/(grid_count*2)
np.linspace(0+blood,1-blood,grid_count)

array([0.0625, 0.1875, 0.3125, 0.4375, 0.5625, 0.6875, 0.8125, 0.9375])

In [ ]:
[[],[],[],[]]

In [33]:
def transform(image_path, center, ltwh, offset, determine):
    image = tf.io.decode_jpeg(tf.io.read_file(image_path))
    object_map_param = [
        (128,(0,36)),
        (64,(37,76)),
        (32,(77,156)),
        (16,(157,316)),
        (8,(317,636)),
        (8,(637,892)),
        (8,(892,1024))]
    object_map = []
    for grid_count, object_determine_size_range in object_map_param:
        smallest, biggest = object_determine_size_range
        mask = 
        
        object_map.append(np.zeros((grid_count,grid_count)))
    return image, *object_map
    
    

In [24]:
class Dataset:
    def __init__(self,label_folder,image_folder,batch_size,image_shape):
        self.label_folder = label_folder
        self.image_folder = image_folder
        self.batch_size = batch_size
        self.image_shape = image_shape
    
    def split(self,validation_ratio,random_state):
        labels = [l.as_posix() for l in pathlib.Path(self.label_folder).iterdir()]
        train_labels, validation_labels = train_test_split(labels,random_state=random_state)
        train_data = self.create_dataset(train_labels,False)
        validation_data = self.create_dataset(validation_labels,True)
        return train_data, validation_data
    
    def create_dataset(self,labels,is_validation):
        dataset = tf.data.Dataset.from_tensor_slices(labels)
        dataset = dataset.map(
            lambda path: tf.py_function(self.decode_json, [path], [tf.string, tf.float64, tf.float64, tf.float64, tf.int32]),
            num_parallel_calls=tf.data.AUTOTUNE
        )
        dataset = dataset.cache()
        if is_validation:
            dataset = dataset.shuffle(buffer_size=5000)
        dataset
        return dataset
        
    def decode_json(self,path):
        label = json.loads(tf.io.read_file(path).numpy())

        rectangle = np.array([b['rectangle']['points'] for b in label['bboxes']])
        polygon = np.array([b['polygon']['points'] for b in label['bboxes']])
        offset = polygon - rectangle

        ltwh = np.array([b['rectangle']['ltwh'] for b in label['bboxes']])
        center = np.array([b['rectangle']['center'] for b in label['bboxes']])

        determine = np.ceil(np.max(ltwh[:,2:]*np.array(self.image_shape),axis=-1)).astype(int)

        image_path = tf.strings.join([self.image_folder,label['ImageName']],'/')
        return image_path, center, ltwh, offset, determine